<a href="https://colab.research.google.com/github/nataliyah123/ibia-Ersilia/blob/main/AutoGloun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Autogluon Example 

In [ ]:
%%capture

!pip install rdkit-pypi

In [ ]:
import pandas as pd
from rdkit import Chem
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
smilescsv = pd.read_csv('dilismiles.csv', names = ['col_smiles'])

In [ ]:
data = pd.read_csv('dili_padel_2d.csv')
dataframe = pd.DataFrame(data)
desc = pd.DataFrame(dataframe.iloc[:, 3:])

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (423,424,425,426,427,428,429,430,431,477,478,479,480,481,482,483,484,485,540,541,542,543,544,545,546,547,548,603,604,605,606,607,608,609,610,611,1012,1013,1050,1052,1074,1083,1084,1121,1123,1145,1217,1218,1219,1220,1222,1223,1224,1226,1227,1229) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Install Autogluon

In [ ]:
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
# Install autogluon (Tutorial based on autogluon==0.1.0)
!pip install autogluon
# Upgrade ipykernel (Necessary for Colab)
!pip install -U ipykernel
# !pip install autogluon==0.0.15 autogluon.tabular "mxnet<2.0.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 10.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
X = TabularDataset('dili_padel_2d.csv')
print(X.head(), X.shape)

               ID         CommonName  Outcome  nAcid  naAromAtom  nAromBond  \
0  AUTOGEN_dili_1    Aurothioglucose        1      0           0          0   
1  AUTOGEN_dili_2  DemecariumBromide        1      0          12         12   
2  AUTOGEN_dili_3   Dextromethorphan        0      0           6          6   
3  AUTOGEN_dili_4         Ergotamine        0      0          15         16   
4  AUTOGEN_dili_5     Beclomethasone        0      0           0          0   

   nB  nN  nO  nS  ...       AMW      WTPT1     WTPT2      WTPT3      WTPT4  \
0   0   0   5   1  ...  8.479683  23.078977  1.923248  15.422349  12.919448   
1   0   4   4   0  ...  6.047814  78.841924  1.971048  24.051089  11.127775   
2   0   1   1   0  ...  6.026525  42.041518  2.102076   6.134966   2.806398   
3   0   5   5   0  ...  7.452100  90.264234  2.099168  30.004012  13.475249   
4   0   0   5   0  ...  7.160883  56.562478  2.020089  15.115392  12.515759   

       WTPT5  WPATH  WPOL  XLogP  Zagreb  
0   0.0

In [ ]:
y = dataframe['Outcome']
print(type(y),y.shape)

<class 'pandas.core.series.Series'> (588,)


In [ ]:
#deleting incomplete cases rows
delrow = ["GalliumNitrate Hydrate","trichloroethylene", "Bromoethanamine", "sodiumbicarbonate", "carbontetrachloride", "Chloroform", "cadmiumchloride", "thioacetamide", "Probucol","dichloroethylene", "Hydrazine", "nitrosamine"]
print(X.iloc[64]['CommonName'], X.shape)
fory = []
ind = X.index
print(ind)
for i in delrow:  
  # print(X[X['CommonName']==i].any)  
  if(X[X['CommonName']==i].any):     
    indexname = X[X['CommonName']==i].index
    what = ind[indexname].tolist()
    fory.append(what[0])    
    X.drop(indexname, inplace=True)
    y.drop(indexname, inplace=True)

print(X.shape, len(delrow),fory, y.shape)

GalliumNitrate Hydrate (588, 1447)
RangeIndex(start=0, stop=588, step=1)
(576, 1447) 12 [64, 84, 85, 94, 105, 106, 296, 301, 313, 435, 554, 555] (576,)


In [ ]:
# deleting columns with incomplete case
X.drop(['BCUTw1l',"BCUTw1h","BCUTc1l","BCUTc1h","BCUTp1l","BCUTp1h","nHdsCH","ndsCH","nsssCH","maxHdsCH","maxsssCH","maxdsCH","SHdsCH","SdsCH","SsssCH","minHdsCH","minsssCH","nHdsCH","nsCH3","nssCH2","ndsCH","nsssCH"], axis=1,inplace=True)
X.shape

(576, 1428)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [ ]:
label = 'Outcome'

In [ ]:
fit_args = {}
fit_args['time_limit'] = 1200
predictor = TabularPredictor(label=label).fit(x_train,presets="best_quality", **fit_args)
performance = predictor.evaluate(x_test)

No path specified. Models will be saved in: "AutogluonModels/ag-20221223_052036/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "AutogluonModels/ag-20221223_052036/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    460
Train Data Columns: 1427
Label Column: Outcome
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Gene

# With Word2Vec

In [ ]:
%%capture
!pip install git+https://github.com/samoturk/mol2vec

In [ ]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence,DfVec, sentences2vec
import numpy as np
from gensim.models import word2vec

In [ ]:
model = word2vec.Word2Vec.load('model_300dim_eric.pkl')
smilescsv['mol'] = smilescsv['col_smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
smilescsv['sentence'] = smilescsv.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
smilescsv['mol2vec'] = [DfVec(x) for x in sentences2vec(smilescsv['sentence'], model, unseen='UNK')]

In [ ]:
x_trainw2v, x_testw2v, y_trainw2v, y_testw2v = train_test_split(X,y,test_size=0.20)

In [ ]:
fit_args = {}
# fit_args['excluded_model_types'] = ['NN', 'FASTAI']
fit_args['time_limit'] = 1200
predictor = TabularPredictor(label=label).fit(x_trainw2v,presets="best_quality", **fit_args)
performance = predictor.evaluate(x_testw2v)

No path specified. Models will be saved in: "AutogluonModels/ag-20221223_054859/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "AutogluonModels/ag-20221223_054859/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    460
Train Data Columns: 1427
Label Column: Outcome
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Gene